In [25]:
import pandas as pd

gs_res = pd.read_csv('output/lgb_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_auc'], ascending=False)
#0.829998
gs_res.head()

,bagging_fraction,bagging_freq,bagging_seed,best_round,boosting,feature_fraction,feature_fraction_seed,gs_timespent,learning_rate,max_depth,metric,min_data_in_leaf,num_leaves,num_threads,objective,pred_timespent,scale_pos_weight,timestamp,train_auc,val_auc
1530224002,0.8,1,23892211,20,gbdt,1.0,23892211,0:00:01,0.10,-1,auc,34,15,8,binary,0:00:02,1,2018-06-28 18:13:22,0.727692,0.724557
1530220352,0.8,1,37460974,8,gbdt,0.9,37460974,0:00:02,0.03,-1,auc,41,31,8,binary,0:00:03,1,2018-06-28 17:12:32,0.726405,0.723331
1530220336,0.6,1,21771510,20,gbdt,1.0,21771510,0:00:08,0.03,-1,auc,24,31,8,binary,0:00:05,1,2018-06-28 17:12:16,0.725626,0.721848


In [32]:
import pandas as pd

gs_nn_res = pd.read_csv('output/nn_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_auc'], ascending=False)
#0.829998
gs_nn_res.head()

,batch_size,best_epoch,cat_emb_outdim,combined_dense_n_layers,dense_units,drop_rate,ep_for_lr,gs_timespent,lr_fin,lr_init,max_ep,nn_seed,num_dense_n_layers,patience,timestamp,trn_acc,trn_loss,val_acc,val_auc,val_loss
1530288201,2048,1,50,1,50,0.2,1,0:00:08,0.005,0.01,1,34341622,1,5,2018-06-29 12:03:21,0.812054,0.442795,0.81569,0.734491,0.430824


In [27]:
from os import listdir

In [30]:
for filename in listdir('output'):
    if '_grid_search' in filename:
        print(filename)

nn_grid_search.csv
lgb_grid_search.csv


In [2]:
gs_res.head(1).T.to_dict()

{1530207552: {'bagging_fraction': 0.5,
  'bagging_freq': 1,
  'bagging_seed': 38625225,
  'best_round': 38,
  'boosting': 'gbdt',
  'cv': True,
  'feature_fraction': 0.9,
  'feature_fraction_seed': 38625225,
  'gs_timespent': '0:00:00',
  'learning_rate': 0.1,
  'max_depth': -1,
  'metric': 'auc',
  'min_data_in_leaf': 27,
  'num_leaves': 31,
  'num_threads': 8,
  'objective': 'binary',
  'pred_timespent': '0:00:00',
  'scale_pos_weight': 1,
  'timestamp': '2018-06-28 13:39:12',
  'val_auc': 0.7091430993869628}}

In [9]:
#gs_res.gs_timespent
gs_res.pred_timespent

1529021613        NaN
1529016426        NaN
1529018698        NaN
1529038836        NaN
1529079354        NaN
1529078964        NaN
1529080397    0:01:21
1529080656    0:01:10
1529005138        NaN
1529078391        NaN
1529014439        NaN
1529014419        NaN
1529014400        NaN
1529005975        NaN
1529014454        NaN
1529007956        NaN
1529006206        NaN
1529014393        NaN
1529014412        NaN
1529014409        NaN
1529007953        NaN
1529005557        NaN
1529014159        NaN
1529006204        NaN
1529014405        NaN
1529005829        NaN
1529014458        NaN
1529005980        NaN
1529010730        NaN
1529014416        NaN
1529014398        NaN
1529010729        NaN
1529014162        NaN
1529005826        NaN
1529014434        NaN
Name: pred_timespent, dtype: object

In [1]:
import pandas as pd

In [2]:
pd.__version__

'0.23.1'

In [23]:
'Model__131'.split('__')[0]

'Model'